# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key
 
 
CITY_OUTPUT_DATA_PATH= "output_data/cities.csv"

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(CITY_OUTPUT_DATA_PATH)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ayorou,14.7308,0.9174,34.10,7,18,4.68,NE,1704814969
1,1,avarua,-21.2078,-159.7750,25.03,83,75,3.09,CK,1704814969
2,2,vysokaya gora,55.9128,49.3017,-14.14,97,92,3.32,RU,1704814970
3,3,zunyi,27.6867,106.9072,9.51,84,76,1.13,CN,1704814723
4,4,west island,-12.1568,96.8225,27.99,83,75,5.14,CC,1704814975


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")

# Display the map
map_plot_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27))
wind_condition = (city_data_df["Wind Speed"] < 4.5)
cloud_condition = (city_data_df["Cloudiness"] == 0)

city_data_fil = city_data_df.loc[(temp_condition & wind_condition & cloud_condition), :]


# Drop any rows with null values
city_final = city_data_fil.dropna()

# Display sample data
city_final

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,pisco,-13.7000,-76.2167,25.01,78,0,3.09,PE,1704815013
135,135,kish,26.5578,54.0194,21.15,46,0,3.09,IR,1704815020
179,179,tamanrasset,22.7850,5.5228,22.95,10,0,2.06,DZ,1704815037
196,196,kapa'a,22.0752,-159.3190,21.18,78,0,2.06,US,1704815042
510,510,swan hill,-35.3378,143.5544,23.38,63,0,1.14,AU,1704814947


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
113,pisco,PE,-13.7000,-76.2167,78,
135,kish,IR,26.5578,54.0194,46,
179,tamanrasset,DZ,22.7850,5.5228,10,
196,kapa'a,US,22.0752,-159.3190,78,
510,swan hill,AU,-35.3378,143.5544,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accomodation.hotel"]
apiKey = geoapify_key

params = {
    "categories": categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
   longitude = hotel_df.loc[index, "Lng"]
   latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   params["filter"] = f'circle:{longitude},{latitude},{radius}'
   params["bias"] = f'proximity:{longitude},{latitude}'
   params["categories"] = f'accommodation.hotel'
    
    # Set base URL
   base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
   name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
   name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
   except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
   print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
kish - nearest hotel: هتل فارابی
tamanrasset - nearest hotel: فندق الأمان
kapa'a - nearest hotel: Pono Kai Resort
swan hill - nearest hotel: Murray Downs Golf Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
113,pisco,PE,-13.7000,-76.2167,78,La Portada
135,kish,IR,26.5578,54.0194,46,هتل فارابی
179,tamanrasset,DZ,22.7850,5.5228,10,فندق الأمان
196,kapa'a,US,22.0752,-159.3190,78,Pono Kai Resort
510,swan hill,AU,-35.3378,143.5544,63,Murray Downs Golf Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)